# <center> Ship Generation

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import pickle
import json

print("Imports loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Imports loaded successfully!
NumPy version: 2.2.6
Pandas version: 2.3.3


In [39]:
# ==============================================================================
# GLOBAL VARIABLES - ADJUST THESE TO CONFIGURE THE SIMULATION
# ==============================================================================

# List of HS codes to include in the simulation (2-digit level)
# These will be matched against hs_codes_mapping.json for names and ship types
HS_CODES_LIST = [i for i in range(1, 99) if i != 77]

# Simulation parameters
CAPACITY_QUANTILE = 0.99  # 99% of ship capacities should be below max capacity
DIRICHLET_CONCENTRATION = 1  # Concentration parameter for Dirichlet distribution (cargo ships only)

# Data directories
DATA_DIR = '../../data/'
TRADE_MATRICES_DIR = DATA_DIR + 'all_trade_matrices/'
CONVERSION_DIR = '../part_2_trade_volume_conversion/trade_volume_conversion_output/'
NETWORK_DIR = '../part_3_network_extraction/network_outputs/'

print("="*70)
print("GLOBAL VARIABLES CONFIGURED")
print("="*70)
print(f"HS Codes selected: {HS_CODES_LIST}")
print(f"Capacity quantile: {CAPACITY_QUANTILE}")
print(f"Dirichlet concentration: {DIRICHLET_CONCENTRATION}")
print("="*70)

GLOBAL VARIABLES CONFIGURED
HS Codes selected: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]
Capacity quantile: 0.99
Dirichlet concentration: 1


In [40]:
# ==============================================================================
# LOAD NETWORK
# ==============================================================================
print("="*70)
print("LOADING NETWORK")
print("="*70)

# Load the Contraction Hierarchies network
with open(f'{NETWORK_DIR}network_contraction_hierarchies.gpickle', 'rb') as f:
    G_ch = pickle.load(f)

print(f"Network loaded: {G_ch.number_of_nodes()} nodes, {G_ch.number_of_edges()} edges")

# Print node type breakdown
port_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'port')
choke_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') == 'choke_point')
network_count = sum(1 for n in G_ch.nodes() if G_ch.nodes[n].get('source') not in ['port', 'choke_point'])

print(f"  Ports: {port_count}")
print(f"  Choke points: {choke_count}")
print(f"  Network nodes: {network_count}")

# Create port-country and country-port mappings
port_to_country = {}
country_to_ports = {}

for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        country = G_ch.nodes[node].get('country')
        
        if port_name and country:
            port_to_country[port_name] = country
            
            if country not in country_to_ports:
                country_to_ports[country] = []
            country_to_ports[country].append(port_name)

print(f"\nPort-Country Mappings:")
print(f"  Unique ports: {len(port_to_country)}")
print(f"  Unique countries: {len(country_to_ports)}")

print(f"\nPorts by country (first 10):")
for country in sorted(country_to_ports.keys())[:10]:
    print(f"  {country}: {len(country_to_ports[country])} ports")

print("="*70)

LOADING NETWORK
Network loaded: 328 nodes, 526 edges
  Ports: 42
  Choke points: 1
  Network nodes: 285

Port-Country Mappings:
  Unique ports: 42
  Unique countries: 22

Ports by country (first 10):
  Albania: 1 ports
  Algeria: 2 ports
  Bosnia And Herzegovina: 1 ports
  Croatia: 2 ports
  Cyprus: 1 ports
  Egypt: 3 ports
  France: 1 ports
  Greece: 1 ports
  Israel: 1 ports
  Italy: 12 ports


In [41]:
# ==============================================================================
# LOAD HS CODE MAPPING FROM JSON
# ==============================================================================
print("="*70)
print("LOADING HS CODE MAPPING")
print("="*70)

# Load the HS codes mapping file
with open(f'{DATA_DIR}hs_codes_mapping.json', 'r') as f:
    hs_codes_mapping_full = json.load(f)

# Filter to only the selected HS codes
HS_CODES = {}
for hs_code in HS_CODES_LIST:
    hs_str = str(hs_code).zfill(2)  # Convert to 2-digit string (e.g., "10")
    if hs_str in hs_codes_mapping_full:
        HS_CODES[hs_code] = hs_codes_mapping_full[hs_str]
    else:
        print(f"  ⚠ Warning: HS code {hs_code} not found in mapping file")

print(f"✓ Loaded {len(HS_CODES)} HS codes from mapping file")
print(f"\nHS Code Configuration:")
for hs_code, info in HS_CODES.items():
    print(f"  HS{hs_code:02d}: {info['name']} → {info['ship_type']}")
print("="*70)

LOADING HS CODE MAPPING
  ⚠ Warning: HS code 98 not found in mapping file
✓ Loaded 96 HS codes from mapping file

HS Code Configuration:
  HS01: Live animals → cargo ship
  HS02: Meat and edible meat offal → cargo ship
  HS03: Fish and crustaceans, molluscs and other aquatic invertebrates → cargo ship
  HS04: Dairy produce; birds eggs; natural honey; edible products of animal origin → cargo ship
  HS05: Products of animal origin, not elsewhere specified or included → cargo ship
  HS06: Live trees and other plants; bulbs, roots and the like; cut flowers → cargo ship
  HS07: Edible vegetables and certain roots and tubers → cargo ship
  HS08: Edible fruit and nuts; peel of citrus fruit or melons → cargo ship
  HS09: Coffee, tea, mate and spices → cargo ship
  HS10: Cereals → bulk carrier
  HS11: Products of the milling industry; malt; starches; inulin; wheat gluten → bulk carrier
  HS12: Oil seeds and oleaginous fruits; miscellaneous grains, seeds and fruit → bulk carrier
  HS13: Lac; gum

In [42]:
# ==============================================================================
# LOAD MERCHANT FLEET DATA AND CREATE SHIP TYPE DISTRIBUTIONS
# ==============================================================================
from scipy import stats
from scipy.optimize import fsolve

print("="*70)
print("LOADING MERCHANT FLEET DATA")
print("="*70)

# Load fleet data
fleet_df = pd.read_csv(f'{DATA_DIR}merchant_fleet_data.csv')
print(f"✓ Loaded merchant fleet data")
print(f"\n{fleet_df.to_string(index=False)}")

# Extract ship type data - mapping our three types to fleet data
ship_type_mapping = {
    'tanker': 'Oil Tanker',  # Using oil tanker as representative for all tankers
    'bulk carrier': 'Bulk Carrier',
    'cargo ship': 'Container Ship'  # Using container ship as representative for cargo ships
}

ship_types_data = {}
for our_type, fleet_type in ship_type_mapping.items():
    row = fleet_df[fleet_df['Ship Type'] == fleet_type].iloc[0]
    ship_types_data[our_type] = {
        'avg_dwt': float(row['Avg. dwt per ship']) * 1000,  # Convert to tons
        'max_dwt': float(row['Max dwt per ship']) * 1000,   # Convert to tons
        'total_dwt': float(row['Deadweight Tons (in thousand tons)']) * 1000,
        'num_ships': int(row['Number of Ships'])
    }

print(f"\n{'='*70}")
print("SHIP TYPE STATISTICS")
print("="*70)
for ship_type, data in ship_types_data.items():
    print(f"\n{ship_type.title()}:")
    print(f"  Average DWT: {data['avg_dwt']:,.0f} tons")
    print(f"  Max DWT: {data['max_dwt']:,.0f} tons")
    print(f"  Total fleet DWT: {data['total_dwt']:,.0f} tons")
    print(f"  Number of ships: {data['num_ships']:,}")

# ==============================================================================
# CREATE SHIP TYPE-SPECIFIC GAMMA DISTRIBUTIONS
# ==============================================================================
print(f"\n{'='*70}")
print("CREATING SHIP TYPE GAMMA DISTRIBUTIONS")
print("="*70)

def find_gamma_params(mean, max_val, quantile=0.99):
    """
    Find Gamma distribution parameters (shape α, scale θ) such that:
    - Mean = α * θ = mean
    - P(X ≤ max_val) = quantile
    
    Returns: (shape, scale)
    """
    def equations(params):
        alpha, theta = params
        # Equation 1: mean constraint
        eq1 = alpha * theta - mean
        # Equation 2: quantile constraint
        eq2 = stats.gamma.cdf(max_val, alpha, scale=theta) - quantile
        return [eq1, eq2]
    
    # Initial guess: high shape for tight distribution
    initial_guess = [10, mean / 10]
    
    try:
        solution = fsolve(equations, initial_guess)
        alpha, theta = solution
        
        # Verify solution is valid
        if alpha > 0 and theta > 0:
            actual_mean = alpha * theta
            actual_quantile = stats.gamma.cdf(max_val, alpha, scale=theta)
            
            # Check if solution is reasonable
            if abs(actual_mean - mean) / mean < 0.01 and abs(actual_quantile - quantile) < 0.01:
                return alpha, theta
    except:
        pass
    
    # Fallback: use CV-based approach
    # If we can't satisfy both constraints, prioritize the mean
    cv = 0.3  # Assume 30% coefficient of variation
    std = mean * cv
    variance = std ** 2
    
    theta = variance / mean
    alpha = mean / theta
    
    return alpha, theta

ship_distributions = {}

print(f"\nQuantile constraint: {CAPACITY_QUANTILE:.2%} of samples below max capacity")
print()

for ship_type, data in ship_types_data.items():
    mean = data['avg_dwt']
    max_val = data['max_dwt']
    
    # Find Gamma parameters using global CAPACITY_QUANTILE
    shape, scale = find_gamma_params(mean, max_val, CAPACITY_QUANTILE)
    
    # Store distribution
    ship_distributions[ship_type] = {
        'shape': shape,
        'scale': scale,
        'mean': mean,
        'max': max_val,
        'std': np.sqrt(shape) * scale
    }
    
    # Verify
    actual_mean = shape * scale
    actual_std = np.sqrt(shape) * scale
    actual_cv = actual_std / actual_mean
    actual_quantile = stats.gamma.cdf(max_val, shape, scale=scale)
    
    print(f"{ship_type.title()}:")
    print(f"  Target mean: {mean:,.0f} tons")
    print(f"  Gamma shape (α): {shape:.4f}")
    print(f"  Gamma scale (θ): {scale:,.2f}")
    print(f"  Actual mean: {actual_mean:,.0f} tons")
    print(f"  Actual std: {actual_std:,.0f} tons")
    print(f"  Actual CV: {actual_cv:.4f}")
    print(f"  P(X ≤ max): {actual_quantile:.4f} (target: {CAPACITY_QUANTILE:.4f})")
    print()

print("="*70)

LOADING MERCHANT FLEET DATA
✓ Loaded merchant fleet data

          Ship Type  Deadweight Tons (in thousand tons)  Number of Ships  Avg. dwt per ship  Max dwt per ship
        Total Fleet                             2360318           109898          21.477352           404.389
         Oil Tanker                              665472            12514          53.178200           323.514
       Bulk Carrier                             1006329            13717          73.363636           404.389
      General Cargo                               85809            20958           4.094332           175.897
     Container Ship                              331567             6372          52.034997           240.739
Other Ship Capacity                              271141            56337           4.812841           155.159

SHIP TYPE STATISTICS

Tanker:
  Average DWT: 53,178 tons
  Max DWT: 323,514 tons
  Total fleet DWT: 665,472,000 tons
  Number of ships: 12,514

Bulk Carrier:
  Average DWT

/var/folders/3d/fbytqdpn2m599fcr4p1zr9_h0000gn/T/ipykernel_7946/2094556666.py:70: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  solution = fsolve(equations, initial_guess)


In [43]:
# ==============================================================================
# LOAD TRADE MATRICES (VALUE-BASED)
# ==============================================================================
print("="*70)
print("LOADING TRADE MATRICES")
print("="*70)

# Load all specified HS code matrices
trade_matrices = {}
for hs_code, description in HS_CODES.items():
    filepath = f'{TRADE_MATRICES_DIR}trade_matrix_all_transport_modes_HS{hs_code}.csv'
    try:
        tm = pd.read_csv(filepath, index_col=0)
        trade_matrices[hs_code] = tm
        print(f"  ✓ HS{hs_code:02d} ({description['name']}, {description['ship_type']}): {tm.shape}")
    except FileNotFoundError:
        print(f"  ✗ HS{hs_code:02d} ({description['name']}): FILE NOT FOUND")
        raise

# Get network countries
network_countries = set(country_to_ports.keys())

# Process matrices: remove 'World' and filter
processed_matrices = {}
for hs_code, tm_raw in trade_matrices.items():
    tm = tm_raw.copy()
    if 'World' in tm.index:
        tm.drop('World', axis=0, inplace=True)
    if 'World' in tm.columns:
        tm.drop('World', axis=1, inplace=True)
    processed_matrices[hs_code] = tm

# Get common countries
first_hs = list(processed_matrices.keys())[0]
trade_countries = set(processed_matrices[first_hs].index)
common_countries = sorted(network_countries & trade_countries)

print(f"\n{'='*70}")
print(f"Filtering to network countries...")
print(f"  Countries in network: {len(network_countries)}")
print(f"  Countries in trade data: {len(trade_countries)}")
print(f"  Countries in BOTH: {len(common_countries)}")

# Filter matrices to common countries
filtered_matrices = {}
for hs_code, tm in processed_matrices.items():
    filtered_matrices[hs_code] = tm.loc[common_countries, common_countries].copy()

# Calculate total trade volume
total_trade_volume = sum(filtered_matrices.values())

print(f"\nTrade statistics (value, network countries only):")
for hs_code, tm in filtered_matrices.items():
    print(f"  HS{hs_code:02d}: ${tm.sum().sum():,.0f}")
print(f"  {'─' * 68}")
print(f"  TOTAL: ${total_trade_volume.sum().sum():,.0f}")
print("="*70)

LOADING TRADE MATRICES
  ✓ HS01 (Live animals, cargo ship): (227, 227)
  ✓ HS02 (Meat and edible meat offal, cargo ship): (227, 227)
  ✓ HS03 (Fish and crustaceans, molluscs and other aquatic invertebrates, cargo ship): (227, 227)
  ✓ HS04 (Dairy produce; birds eggs; natural honey; edible products of animal origin, cargo ship): (227, 227)
  ✓ HS05 (Products of animal origin, not elsewhere specified or included, cargo ship): (227, 227)
  ✓ HS06 (Live trees and other plants; bulbs, roots and the like; cut flowers, cargo ship): (227, 227)
  ✓ HS07 (Edible vegetables and certain roots and tubers, cargo ship): (227, 227)
  ✓ HS08 (Edible fruit and nuts; peel of citrus fruit or melons, cargo ship): (227, 227)
  ✓ HS09 (Coffee, tea, mate and spices, cargo ship): (227, 227)
  ✓ HS10 (Cereals, bulk carrier): (227, 227)
  ✓ HS11 (Products of the milling industry; malt; starches; inulin; wheat gluten, bulk carrier): (227, 227)
  ✓ HS12 (Oil seeds and oleaginous fruits; miscellaneous grains, seeds

In [44]:
# ==============================================================================
# LOAD CONVERSION FACTORS AND CREATE WEIGHT MATRICES
# ==============================================================================
print("="*70)
print("LOADING CONVERSION FACTORS")
print("="*70)

# Load ONLY general conversion factors
with open(f'{CONVERSION_DIR}trade_volume_conversion_general.json', 'r') as f:
    conversion_general_raw = json.load(f)

# Convert to proper format
conversion_general = {}
for hs_str, data in conversion_general_raw.items():
    conversion_general[int(hs_str)] = float(data['conversion_factor'])

print(f"✓ Loaded {len(conversion_general)} general conversion factors")

# Display factors for our HS codes
print(f"\nGeneral conversion factors (USD/ton):")
for hs_code, description in HS_CODES.items():
    if hs_code in conversion_general:
        print(f"  HS{hs_code:02d}: ${conversion_general[hs_code]:.2f}/ton")

def get_conversion_factor(hs_code):
    """Get general conversion factor (USD/ton) for an HS code"""
    return conversion_general.get(hs_code, None)

print(f"\n{'='*70}")
print("CREATING WEIGHT MATRICES")
print("="*70)

weight_matrices = {}

for hs_code, description in HS_CODES.items():
    value_matrix = filtered_matrices[hs_code]
    weight_matrix = pd.DataFrame(
        np.zeros_like(value_matrix.values),
        index=value_matrix.index,
        columns=value_matrix.columns
    )
    
    # Convert each cell using GENERAL conversion factor
    cf = get_conversion_factor(hs_code)
    if cf and cf > 0:
        # Apply the same conversion factor to all country pairs
        weight_matrix = value_matrix / cf
    
    weight_matrices[hs_code] = weight_matrix
    
    total_val = value_matrix.sum().sum()
    total_wt = weight_matrix.sum().sum()
    avg_price = total_val / total_wt if total_wt > 0 else 0
    
    print(f"  HS{hs_code:02d}: {total_wt:,.0f} tons (${total_val:,.0f}, avg ${avg_price:.2f}/ton)")

# Calculate totals
total_trade_weight = sum(weight_matrices.values())
total_weight_sum = total_trade_weight.sum().sum()

print(f"\n{'='*70}")
print(f"TOTAL WEIGHT: {total_weight_sum:,.0f} metric tons")
print(f"TOTAL VALUE:  ${total_trade_volume.sum().sum():,.0f}")
print(f"Overall avg:  ${total_trade_volume.sum().sum() / total_weight_sum:.2f}/ton")
print("="*70)

LOADING CONVERSION FACTORS
✓ Loaded 96 general conversion factors

General conversion factors (USD/ton):
  HS01: $3881.79/ton
  HS02: $3427.68/ton
  HS03: $5002.38/ton
  HS04: $2775.20/ton
  HS05: $1876.12/ton
  HS06: $3618.38/ton
  HS07: $930.97/ton
  HS08: $1558.54/ton
  HS09: $4401.19/ton
  HS10: $335.02/ton
  HS11: $704.15/ton
  HS12: $642.84/ton
  HS13: $8245.67/ton
  HS14: $294.91/ton
  HS15: $1252.81/ton
  HS16: $5590.43/ton
  HS17: $801.11/ton
  HS18: $4507.36/ton
  HS19: $3172.71/ton
  HS20: $1834.88/ton
  HS21: $4508.88/ton
  HS22: $145.58/ton
  HS23: $636.38/ton
  HS24: $13514.18/ton
  HS25: $94.00/ton
  HS26: $176.74/ton
  HS27: $508.62/ton
  HS28: $1041.94/ton
  HS29: $2129.12/ton
  HS30: $118132.96/ton
  HS31: $434.43/ton
  HS32: $3774.13/ton
  HS33: $13817.17/ton
  HS34: $2416.92/ton
  HS35: $4435.13/ton
  HS36: $5438.08/ton
  HS37: $18580.18/ton
  HS38: $3098.17/ton
  HS39: $2596.91/ton
  HS40: $3729.27/ton
  HS41: $3325.42/ton
  HS42: $21392.71/ton
  HS43: $86969.21/to

In [45]:
# ==============================================================================
# CALCULATE WEIGHTED AVERAGE CAPACITY AND N_SHIPS
# ==============================================================================
print("="*70)
print("CALCULATING N_SHIPS")
print("="*70)

# Calculate weight by ship type
print(f"\nCalculating weight distribution by ship type...")
weight_by_ship_type = {}

for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
    total_weight = 0
    for hs_code, info in HS_CODES.items():
        if info['ship_type'] == ship_type:
            total_weight += weight_matrices[hs_code].sum().sum()
    weight_by_ship_type[ship_type] = total_weight
    print(f"  {ship_type.title()}: {total_weight:,.0f} tons")

total_weight_all = sum(weight_by_ship_type.values())
print(f"  Total: {total_weight_all:,.0f} tons")

# Calculate weighted average capacity
weighted_avg_capacity = 0
for ship_type, weight in weight_by_ship_type.items():
    proportion = weight / total_weight_all
    avg_capacity = ship_distributions[ship_type]['mean']
    weighted_avg_capacity += proportion * avg_capacity
    print(f"\n  {ship_type.title()}: {proportion:.4f} × {avg_capacity:,.0f} = {proportion * avg_capacity:,.0f} tons")

print(f"\nWeighted average capacity: {weighted_avg_capacity:,.0f} tons")

# Calculate N_SHIPS dynamically
N_SHIPS = int(np.ceil(total_weight_all / weighted_avg_capacity))

print(f"\n{'='*70}")
print(f"DYNAMIC N_SHIPS CALCULATION")
print(f"{'='*70}")
print(f"  Total weight: {total_weight_all:,.0f} tons")
print(f"  Weighted avg capacity: {weighted_avg_capacity:,.0f} tons")
print(f"  N_SHIPS (calculated): {N_SHIPS:,}")
print(f"\n  This ensures realistic ship sizes based on actual fleet data!")
print("="*70)

CALCULATING N_SHIPS

Calculating weight distribution by ship type...
  Tanker: 255,855,597 tons
  Bulk Carrier: 141,226,923 tons
  Cargo Ship: 111,071,809 tons
  Total: 508,154,330 tons

  Tanker: 0.5035 × 53,178 = 26,775 tons

  Bulk Carrier: 0.2779 × 73,364 = 20,389 tons

  Cargo Ship: 0.2186 × 52,035 = 11,374 tons

Weighted average capacity: 58,538 tons

DYNAMIC N_SHIPS CALCULATION
  Total weight: 508,154,330 tons
  Weighted avg capacity: 58,538 tons
  N_SHIPS (calculated): 8,681

  This ensures realistic ship sizes based on actual fleet data!


In [46]:
# ==============================================================================
# COMPUTE ROUTES
# ==============================================================================
print("="*70)
print("COMPUTING SHORTEST PATHS BETWEEN ALL COUNTRY PAIRS")
print("="*70)

# Create port name to node mapping
port_name_to_node = {}
for node in G_ch.nodes():
    if G_ch.nodes[node].get('source') == 'port':
        port_name = G_ch.nodes[node].get('portName')
        if port_name:
            port_name_to_node[port_name] = node

country_pair_routes = {}
n_pairs = len(common_countries) * (len(common_countries) - 1)

for origin_country in tqdm(common_countries, desc="Computing routes"):
    for dest_country in common_countries:
        if origin_country == dest_country:
            continue
        
        origin_ports = country_to_ports.get(origin_country, [])
        dest_ports = country_to_ports.get(dest_country, [])
        
        if not origin_ports or not dest_ports:
            continue
        
        best_path = None
        best_length = float('inf')
        best_origin_port = None
        best_dest_port = None
        
        # Try all port combinations
        for o_port in origin_ports:
            for d_port in dest_ports:
                o_node = port_name_to_node.get(o_port)
                d_node = port_name_to_node.get(d_port)
                
                if o_node is None or d_node is None:
                    continue
                
                try:
                    path = nx.shortest_path(G_ch, o_node, d_node, weight='length')
                    path_length = sum(G_ch[path[i]][path[i+1]].get('length', 0) 
                                    for i in range(len(path)-1))
                    
                    if path_length < best_length:
                        best_length = path_length
                        best_path = path
                        best_origin_port = o_port
                        best_dest_port = d_port
                except (nx.NetworkXNoPath, nx.NodeNotFound):
                    continue
        
        if best_path:
            country_pair_routes[(origin_country, dest_country)] = {
                'path': best_path,
                'length': best_length,
                'origin_port': best_origin_port,
                'dest_port': best_dest_port
            }

print(f"\n✓ Found routes for {len(country_pair_routes)} country pairs (out of {n_pairs})")
print("="*70)

COMPUTING SHORTEST PATHS BETWEEN ALL COUNTRY PAIRS


Computing routes: 100%|██████████| 20/20 [00:00<00:00, 41.99it/s]


✓ Found routes for 380 country pairs (out of 380)


In [47]:
# ==============================================================================
# GENERATE SHIPS WITH REALISTIC SHIP TYPES AND CAPACITIES
# ==============================================================================
print("="*70)
print(f"GENERATING {N_SHIPS} SHIPS WITH REALISTIC SHIP TYPES")
print("="*70)

# Sample origin-destination pairs based on WEIGHT probabilities
n_countries = len(common_countries)

# Calculate total trade weight matrix (for country pair sampling)
trade_prob = (total_trade_weight / total_weight_sum).values.flatten()
trade_prob = trade_prob / trade_prob.sum()

pair_indices = np.random.choice(n_countries * n_countries, size=N_SHIPS, p=trade_prob)

# Pre-calculate country-pair-specific proportions BY SHIP TYPE
print(f"\nPre-calculating country-pair trade proportions by ship type...")

country_pair_ship_type_proportions = {}

for origin_country in common_countries:
    for dest_country in common_countries:
        if origin_country == dest_country:
            continue
        
        # Calculate weight by ship type for this country pair
        pair_weight_by_type = {}
        for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
            type_weight = 0
            for hs_code, info in HS_CODES.items():
                if info['ship_type'] == ship_type:
                    type_weight += weight_matrices[hs_code].loc[origin_country, dest_country]
            pair_weight_by_type[ship_type] = type_weight
        
        total_pair_weight = sum(pair_weight_by_type.values())
        
        if total_pair_weight > 0:
            # Calculate ship type sampling probabilities (weight / avg_capacity)
            ship_type_probs = {}
            for ship_type, weight in pair_weight_by_type.items():
                avg_cap = ship_distributions[ship_type]['mean']
                ship_type_probs[ship_type] = weight / avg_cap
            
            # Normalize
            total_prob = sum(ship_type_probs.values())
            if total_prob > 0:
                ship_type_probs = {k: v / total_prob for k, v in ship_type_probs.items()}
            else:
                # Fallback: equal probabilities
                ship_type_probs = {k: 1/3 for k in ship_type_probs.keys()}
        else:
            # No trade for this pair, use global proportions
            ship_type_probs = {}
            for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
                weight = weight_by_ship_type[ship_type]
                avg_cap = ship_distributions[ship_type]['mean']
                ship_type_probs[ship_type] = weight / avg_cap
            total_prob = sum(ship_type_probs.values())
            ship_type_probs = {k: v / total_prob for k, v in ship_type_probs.items()}
        
        # Calculate HS code proportions for each ship type
        hs_proportions_by_type = {}
        for ship_type in ['tanker', 'bulk carrier', 'cargo ship']:
            hs_weights = []
            hs_codes_for_type = [hs for hs, info in HS_CODES.items() if info['ship_type'] == ship_type]
            
            for hs_code in hs_codes_for_type:
                hs_weights.append(weight_matrices[hs_code].loc[origin_country, dest_country])
            
            total_type_weight = sum(hs_weights)
            if total_type_weight > 0:
                hs_proportions = [w / total_type_weight for w in hs_weights]
            else:
                # Fallback: equal proportions
                hs_proportions = [1/len(hs_codes_for_type) if len(hs_codes_for_type) > 0 else 0 
                                 for _ in hs_codes_for_type]
            
            hs_proportions_by_type[ship_type] = {
                'hs_codes': hs_codes_for_type,
                'proportions': hs_proportions
            }
        
        country_pair_ship_type_proportions[(origin_country, dest_country)] = {
            'ship_type_probs': ship_type_probs,
            'hs_proportions': hs_proportions_by_type
        }

print(f"✓ Calculated proportions for {len(country_pair_ship_type_proportions)} country pairs")

# Show example
print(f"\nExample: Albania → Algeria")
if ('Albania', 'Algeria') in country_pair_ship_type_proportions:
    example = country_pair_ship_type_proportions[('Albania', 'Algeria')]
    print(f"  Ship type probabilities:")
    for ship_type, prob in example['ship_type_probs'].items():
        print(f"    {ship_type}: {prob:.4f}")

# ==============================================================================
# GENERATE SHIPS
# ==============================================================================
print(f"\n{'='*70}")
print("GENERATING SHIPS")
print("="*70)

ship_data_list = []
ship_type_counts = {'tanker': 0, 'bulk carrier': 0, 'cargo ship': 0}

for ship_id, pair_idx in tqdm(enumerate(pair_indices), total=N_SHIPS, desc="Creating ships"):
    origin_idx = pair_idx // n_countries
    dest_idx = pair_idx % n_countries
    
    origin_country = common_countries[origin_idx]
    dest_country = common_countries[dest_idx]
    
    if origin_country == dest_country:
        continue
    if (origin_country, dest_country) not in country_pair_routes:
        continue
    
    # Get country-pair proportions
    pair_data = country_pair_ship_type_proportions[(origin_country, dest_country)]
    
    # STAGE 1: Sample ship type
    ship_types = list(pair_data['ship_type_probs'].keys())
    ship_type_probs_list = [pair_data['ship_type_probs'][st] for st in ship_types]
    
    # Normalize to handle floating point errors
    ship_type_probs_list = np.array(ship_type_probs_list)
    ship_type_probs_list = ship_type_probs_list / ship_type_probs_list.sum()
    
    ship_type = np.random.choice(ship_types, p=ship_type_probs_list)
    ship_type_counts[ship_type] += 1
    
    # STAGE 2: Sample ship capacity from type-specific Gamma distribution
    shape = ship_distributions[ship_type]['shape']
    scale = ship_distributions[ship_type]['scale']
    max_cap = ship_distributions[ship_type]['max']
    
    ship_capacity = np.random.gamma(shape=shape, scale=scale)
    # Apply cutoff at max capacity
    ship_capacity = min(ship_capacity, max_cap)
    
    # STAGE 3: Sample cargo based on ship type
    if ship_type in ['tanker', 'bulk carrier']:
        # Single HS code
        hs_data = pair_data['hs_proportions'][ship_type]
        hs_codes_available = hs_data['hs_codes']
        hs_probs = hs_data['proportions']
        
        if len(hs_codes_available) == 0 or sum(hs_probs) == 0:
            continue  # Skip if no HS codes available for this ship type
        
        # Normalize probabilities
        hs_probs = np.array(hs_probs)
        hs_probs = hs_probs / hs_probs.sum()
        
        # Sample one HS code
        selected_hs = np.random.choice(hs_codes_available, p=hs_probs)
        
        # Full capacity goes to this HS code
        cargo_by_hs_weight = {hs: 0.0 for hs in HS_CODES.keys()}
        cargo_by_hs_weight[selected_hs] = ship_capacity
        
    else:  # cargo ship
        # Multiple HS codes with Dirichlet
        hs_data = pair_data['hs_proportions'][ship_type]
        hs_codes_available = hs_data['hs_codes']
        hs_probs = hs_data['proportions']
        
        if len(hs_codes_available) == 0:
            continue
        
        # Use Dirichlet with concentration based on proportions (using global parameter)
        alpha = np.maximum(np.array(hs_probs) * DIRICHLET_CONCENTRATION, 1e-10)
        cargo_proportions = np.random.dirichlet(alpha)
        
        # Distribute capacity
        cargo_by_hs_weight = {hs: 0.0 for hs in HS_CODES.keys()}
        for i, hs_code in enumerate(hs_codes_available):
            cargo_by_hs_weight[hs_code] = ship_capacity * cargo_proportions[i]
    
    # Convert weights to values
    cargo_by_hs_value = {}
    total_value = 0.0
    
    for hs_code, cargo_weight in cargo_by_hs_weight.items():
        if cargo_weight > 0:
            conv_factor = get_conversion_factor(hs_code)
            cargo_value = cargo_weight * conv_factor if (conv_factor and conv_factor > 0) else 0.0
        else:
            cargo_value = 0.0
        cargo_by_hs_value[hs_code] = cargo_value
        total_value += cargo_value
    
    # Build ship data
    ship_data = {
        'ship_id': ship_id,
        'origin_country': origin_country,
        'dest_country': dest_country,
        'ship_type': ship_type,
        'cargo_total_weight': float(ship_capacity),
        'cargo_total_value': float(total_value)
    }
    
    # Add HS code cargo fields
    for hs_code in HS_CODES.keys():
        ship_data[f'cargo_hs{hs_code}_weight'] = float(cargo_by_hs_weight[hs_code])
        ship_data[f'cargo_hs{hs_code}_value'] = float(cargo_by_hs_value[hs_code])
    
    ship_data_list.append(ship_data)

print(f"\n✓ Created {len(ship_data_list)} ships")

# Statistics
print(f"\nShip type distribution:")
for ship_type, count in ship_type_counts.items():
    proportion = count / len(ship_data_list) if len(ship_data_list) > 0 else 0
    avg_cap = ship_distributions[ship_type]['mean']
    print(f"  {ship_type.title()}: {count:,} ships ({proportion:.2%})")
    print(f"    Expected ships (weight/capacity): {weight_by_ship_type[ship_type] / avg_cap:,.0f}")

weights = [s['cargo_total_weight'] for s in ship_data_list]
values = [s['cargo_total_value'] for s in ship_data_list]

print(f"\nCargo statistics:")
print(f"  Weight - Mean: {np.mean(weights):,.0f} tons, Std: {np.std(weights):,.0f} tons")
print(f"  Value  - Mean: ${np.mean(values):,.0f}, Std: ${np.std(values):,.0f}")
print(f"  Total weight: {sum(weights):,.0f} tons")
print(f"  Total value: ${sum(values):,.0f}")

# Verify by HS code
print(f"\nCargo by HS code:")
for hs_code, info in HS_CODES.items():
    weight = sum(s[f'cargo_hs{hs_code}_weight'] for s in ship_data_list)
    value = sum(s[f'cargo_hs{hs_code}_value'] for s in ship_data_list)
    expected_weight = weight_matrices[hs_code].sum().sum()
    print(f"  HS{hs_code:02d} ({info['name']}, {info['ship_type']}):")
    print(f"    Weight: {weight:,.0f} tons (expected: {expected_weight:,.0f}, ratio: {weight/expected_weight:.4f})")

print("="*70)

GENERATING 8681 SHIPS WITH REALISTIC SHIP TYPES

Pre-calculating country-pair trade proportions by ship type...
✓ Calculated proportions for 380 country pairs

Example: Albania → Algeria
  Ship type probabilities:
    tanker: 0.5424
    bulk carrier: 0.2170
    cargo ship: 0.2406

GENERATING SHIPS


Creating ships: 100%|██████████| 8681/8681 [00:01<00:00, 6660.38it/s]



✓ Created 8681 ships

Ship type distribution:
  Tanker: 4,491 ships (51.73%)
    Expected ships (weight/capacity): 4,811
  Bulk Carrier: 2,112 ships (24.33%)
    Expected ships (weight/capacity): 1,925
  Cargo Ship: 2,078 ships (23.94%)
    Expected ships (weight/capacity): 2,135

Cargo statistics:
  Weight - Mean: 57,675 tons, Std: 20,495 tons
  Value  - Mean: $77,295,939, Std: $316,050,879
  Total weight: 500,673,405 tons
  Total value: $671,006,043,815

Cargo by HS code:
  HS01 (Live animals, cargo ship):
    Weight: 699,800 tons (expected: 799,443, ratio: 0.8754)
  HS02 (Meat and edible meat offal, cargo ship):
    Weight: 2,008,861 tons (expected: 1,759,197, ratio: 1.1419)
  HS03 (Fish and crustaceans, molluscs and other aquatic invertebrates, cargo ship):
    Weight: 674,605 tons (expected: 1,208,854, ratio: 0.5581)
  HS04 (Dairy produce; birds eggs; natural honey; edible products of animal origin, cargo ship):
    Weight: 2,335,346 tons (expected: 2,075,587, ratio: 1.1251)
  HS

In [48]:
# ==============================================================================
# EXPORT SHIP DATA
# ==============================================================================
print("="*70)
print("EXPORTING SHIP DATA")
print("="*70)

# Create output directory if needed
Path('simulation_output_data').mkdir(exist_ok=True)

# Export to CSV
ship_df = pd.DataFrame(ship_data_list)
ship_df.to_csv('simulation_output_data/simulation_ship_data.csv', index=False)

print(f"✓ Exported simulation_ship_data.csv")
print(f"  Ships: {len(ship_df)}")
print(f"  Columns: {len(ship_df.columns)}")
print(f"  Size: {ship_df.memory_usage(deep=True).sum() / 1024:.1f} KB")

print(f"\nColumn structure:")
print(f"  - ship_id, origin_country, dest_country, ship_type")
print(f"  - cargo_total_weight, cargo_total_value")
print(f"  - cargo_hs{{XX}}_weight, cargo_hs{{XX}}_value (for each HS code)")

print(f"\n{'='*70}")
print("SHIP GENERATION COMPLETE!")
print("="*70)
print(f"\nOutput file:")
print(f"  simulation_output_data/simulation_ship_data.csv")
print(f"\nThis file contains all {len(ship_df)} ships with:")
print(f"  ✓ Realistic ship types (tanker, bulk carrier, cargo ship)")
print(f"  ✓ Realistic capacities (Gamma distributions with 99% < max capacity)")
print(f"  ✓ Single-commodity ships (tankers, bulk carriers)")
print(f"  ✓ Multi-commodity ships (cargo ships with Dirichlet)")
print(f"  ✓ Country-pair-specific trade proportions preserved")
print(f"  ✓ Both weight (tons) and value (USD) for each commodity")
print(f"\nNext step:")
print(f"  Run Mediterranean_Model.ipynb to simulate ship movement")
print("="*70)

EXPORTING SHIP DATA


✓ Exported simulation_ship_data.csv
  Ships: 8681
  Columns: 198
  Size: 15125.6 KB

Column structure:
  - ship_id, origin_country, dest_country, ship_type
  - cargo_total_weight, cargo_total_value
  - cargo_hs{XX}_weight, cargo_hs{XX}_value (for each HS code)

SHIP GENERATION COMPLETE!

Output file:
  simulation_output_data/simulation_ship_data.csv

This file contains all 8681 ships with:
  ✓ Realistic ship types (tanker, bulk carrier, cargo ship)
  ✓ Realistic capacities (Gamma distributions with 99% < max capacity)
  ✓ Single-commodity ships (tankers, bulk carriers)
  ✓ Multi-commodity ships (cargo ships with Dirichlet)
  ✓ Country-pair-specific trade proportions preserved
  ✓ Both weight (tons) and value (USD) for each commodity

Next step:
  Run Mediterranean_Model.ipynb to simulate ship movement
